# LSE Machine Learning: Practical Applications
## Module 4 Unit 2 IDE Activity (Practice) | Practical application of shrinkage methods in R
In this IDE activity, you have the opportunity to practise executing shrinkage methods on a data set in R. You will specifically learn about ridge regression and the lasso.

### Step 1: Load the relevant packages

To execute ridge regression on the credit card data set, the tidyverse, glmnet, caret, ISLR, and psych packages are installed.

In [ ]:
# Set the seed and load packages required
set.seed(123)
library(tidyverse)
library(glmnet)
library(caret)     
library(ISLR)
library(psych)

### Step 2: Load and prepare the data

This example uses the credit card data set. The data is loaded from the ISLR package.
Once the data has been loaded, it must be manipulated, as the glmnet package requires separate predictor and predicted variables. For this data manipulation process, the predicted variable, *y*, needs to be separated from the predictor variables, *x*. In this example, _**balance**_ is the variable that must be predicted. To separate the variables, use the `select` function.

Once the variables are separated, dummy variables need to be created. In this case, print the predictor variables in a matrix format for the glmnet package. Use the `as.matrix` function for the predicted variable and the `model.matrix` function for the predictor variables.The `model.matrix` function also creates the dummy variables that were created manually in the previous unit.

With the data prepared, fit the models onto the data.


In [ ]:
# Load and prepare the data
Credit <- Credit
Credit <- select(Credit,-c(ID)) # Remove the ID
Credit <- na.omit(Credit)         # Make sure that there are no NA values

Split the data into separate predicted and predictor values. Ensure that the categorical variables are transformed using the `model.matrix` function, and scale the predicted data.

In [ ]:
y <- Credit %>% select(Balance) %>% scale(center = TRUE, scale = FALSE) %>% as.matrix()
x <- model.matrix( ~ . - 1, select(Credit,-c(Balance)))

### Step 3: Execute shrinkage methods

#### Ridge regression method
Apply the ```cv.glmnet``` function to estimate the performance of the model across various values of the shrinkage estimator, lambda. This package fits generalised linear models via penalised maximum likelihood, and can be used for a number of different models. In this case, it is used for both ridge regression and the lasso, starting with the ridge regression example.

Add the *x* and *y* parameters, and set the family to Gaussian, as it is a regression problem with Gaussian errors (specifically, *y* is Gaussian). Note that the Gaussian family is the default, so this parameter is omitted. To fit a ridge regression, alpha is set to 0, and to fit a lasso model, alpha is set to 1.

In [ ]:
# Execute 10-fold cross-validation to select the optimal lambda
lambdas_to_try <- 10^seq(0, 10, length.out = 100)

# Set alpha to 0 to implement ridge regression
ridge_cv <- cv.glmnet(x, y, alpha = 0, lambda = lambdas_to_try,
                      standardize = TRUE, nfolds = 10)

In [ ]:
# Plot the results
plot(ridge_cv)

In [ ]:
# Determine the best cross-validated lambda for the ridge regression example
lambda_cv <- ridge_cv$lambda.min

# Fit the final model, and calculate RSS and multiple R-squared values
model_cv <- glmnet(x, y, alpha = 0, lambda = lambda_cv, standardize = TRUE)
y_hat_cv <- predict(model_cv, x)
ssr_cv <- t(y - y_hat_cv) %*% (y - y_hat_cv)
rsq_ridge_cv <- cor(y, y_hat_cv)^2

In [ ]:
# Determine how the coefficients are shrunk by the increasing lambda
res <- glmnet(x, y, alpha = 0, lambda = lambdas_to_try, standardize = FALSE)
plot(res, xvar = "lambda")
legend("bottomright", lwd = 1, col = 1:6, legend = colnames(x), cex = .7)

Notice how the increasing lambda shrinks the coefficients towards 0. The lines in the graph represent the coefficients per variable for the different lambdas. As the lambda increases, coefficients are shrunk towards 0, but never to 0 exactly. 

#### The lasso method

The lasso method is executed similarly to ridge regression. In the next code cell, 10-fold cross-validation is executed to select the optimal lambda, followed by the application of the lasso method, before plotting the cross-validation results. Note that alpha is set to 1 in this method.

In [ ]:
# Execute 10-fold cross-validation to select the optimal lambda
lambdas_to_try <- 10^seq(0, 10, length.out = 100)

# Perform the lasso method by setting alpha = 1
lasso_cv <- cv.glmnet(x, y, alpha = 1, lambda = lambdas_to_try,
                      standardize = TRUE, nfolds = 10)
# Plot the results
plot(lasso_cv)

In [ ]:
# Determine the best cross-validated lambda for the lasso example
lambda_cv <- lasso_cv$lambda.min

# Fit the final model, and calculate RSS and multiple R-squared values
model_cv2 <- glmnet(x, y, alpha = 1, lambda = lambda_cv, standardize = TRUE)
y_hat_cv2 <- predict(model_cv2, x)
ssr_cv2 <- t(y - y_hat_cv2) %*% (y - y_hat_cv2)
rsq_lasso_cv2 <- cor(y, y_hat_cv2)^2

# Determine how the coefficients are shrunk by the increasing lambda
res2 <- glmnet(x, y, alpha = 1, lambda = lambdas_to_try, standardize = FALSE)
plot(res2, xvar = "lambda")
legend("bottomright", lwd = 1, col = 1:6, legend = colnames(x), cex = .7)

Notice how the higher lambda results in more coefficients being shrunk towards 0, some to exactly 0.

Once the estimation is complete, the R-squared values are calculated, and the coefficient and MSE plots for the two models are repeated for easier comparison.

In [ ]:
# Calculate R-squared values for the different models for comparison
rsq <- cbind("R-squared" = c(rsq_ridge_cv, rsq_lasso_cv2))
rownames(rsq) <- c("ridge cross-validated", "lasso cross_validated")
print(rsq)

Repeat the plot of solution paths for both ridge regression and the lasso for easier comparison between the different models.

In [ ]:
# Repeat the plots of solution paths for comparison
par(mfrow=c(2,2))

# Ridge - top left
plot(res, xvar = "lambda")
legend("bottomright", lwd = 1, col = 1:6, legend = colnames(x), cex = .7)

# Ridge - top right
plot(ridge_cv, main="Ridge",cex=0.7)

# Lasso - bottom left
plot(res2, xvar = "lambda")
legend("bottomright", lwd = 1, col = 1:6, legend = colnames(x), cex = .7)

# Lasso - bottom right
plot(lasso_cv, main="LASSO",cex=0.7)

The top-left graph plots the solution path for ridge regression estimated coefficients. As lambda increases, the estimated coefficients shrink closer to 0. The bottom-left graph plots the solution path for the lasso estimates. As lambda increases, more coefficients shrink to exactly 0. The top-right graph plots the cross-validated error versus the log(lambda). The optimal lambda, which minimises the error, and the lambda selected by applying the one standard error rule, suggest that all 12 coefficients are non-zero. For the lasso in the bottom-right graph, the optimal lambda suggests 11 variables are selected, but lambda(1se) only selects 6 variables.

#### Explore further: Obtain coefficients and use for prediction
Instead of performing separate calculations or attempting to make inferences from diagrams, the glmnet package can be used to obtain lambda at the local extrema, or the minimum mean cross-validated error, and the most regularised model. This is done using the `ridge_cv$lambda.min` function and the `ridge_cv$lambda.1se` function.

In [ ]:
# Obtain the lambda 1 standard deviation away from the local extrema
lambda_cv2 <- ridge_cv$lambda.1se # Can use "min" instead of "1se" to obtain the minimum value

# Display the model coefficients
coef(ridge_cv, s="lambda.1se")

# Predict using the cross-validated model
predict(ridge_cv, newx = x[1:10,], s = "lambda.1se")

# Predict using the model fitted on all the data
predict(model_cv, newx = x[1:10,], s = "lambda.1se")

In [ ]:
# Obtain the lambda 1 standard deviation away from the local extrema
lambda_cv2 <- lasso_cv$lambda.1se # Can use "min" instead of "1se" to obtain the minimum value

# Display the model coefficients
coef(lasso_cv, s="lambda.1se")

# Predict using the cross-validated model
predict(lasso_cv, newx = x[1:10,], s = "lambda.1se")

# Predict using the model fitted on all the data
predict(model_cv2, newx = x[1:10,], s = "lambda.1se")

Unlike the ridge regression method, some of the variables in the data are reduced to exactly 0, which shows how the lasso reduces the number of variables. This variable selection process is one of the major advantages of the lasso.